# Step by Step TensorFlow Object Detection API Tutorial
------------------------------------------------------------

## Installation Step


#####  - Tensorflow-script-model
    - git clone https://github.com/tensorflow/models.git
    - extract models
    - buat folder dengan nama 'workshop_dsw'
    - copy dan paste folder 'models' ke dalam folder 'tensorflow_dsw'
        
##### - Akses sebagai global variabel
- Linux
        
        ```
        #command-line :
        export PYTHONPATH="/home/linux/workshop_dsw/models/research:/home/nodeflux/models/research/slim
        # atau bashrc :
        export PYTHONPATH="/home/linux/workshop_dsw/models/research:/home/nodeflux/models/research/slim:$PYTHONPATH
        ```        
- Windows 
        Go to System -> Advanced system settings -> Environment Variables -> New, and add a variable with the name PYTHON_PATH and these values. 
        - Ubah 'PYTHON_PATH' menjadi PYTHONPATH
        - Tambahkan %PYTHONPATH% di system varibles > PATH dan di user variables > PATH
    ![](path_win.png)

##### -Tensorflow

```
         # CPU: 
         pip install tensorflow         
         # GPU: 
         pip install tensorflow-gpu
         # jika menggunakan anaconda
         conda install tensorflow==1.5.0

```
##### - cocoAPI

```
       pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
```
##### - Protobuf
- Linux
    
    ```    
        $ sudo apt-get install protobuf-compiler

        # From workshop_dsw/models/research/        
        $ cd workshop_dsw/models/research
        $ protoc object_detection/protos/*.proto --python_out=.4
        
     ```
        
- Windows    
    - Download [Windows v3.4.0 release “protoc-3.4.0-win32.zip”](https://github.com/google/protobuf/releases/download/v3.4.0/protoc-3.4.0-win32.zip) 
    - Extract the protoc-3.4.0-win32.zip di program files
    - cd path\to\models\research
    - Execute the protobuf compile
            ```
                “C:\Program Files\protoc-3.4.0-win32\bin\protoc.exe” 
                object_detection/protos/*.proto --python_out=.

             ```
        
##### - Download Pretrained models
- checkpoint models [ssd_mobilenet_v1_coco](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)
- label map [label_ssd_mobilenet_v1_coco](    https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt)
    
---------------------------------------

### OUTLINE :
1. Import Libararies
2. Download Model
3. Download Label
4. Inisialisasi memory dan model tensorflow (frozen grap)
5. Inisialisasi Label Map
6. Deteksi


--------------------------------------------------------------------
## Tahap 1 : Import Libararies


In [1]:

'''import library yag dibutuhakan'''
import tensorflow as tf
import os
import numpy as np
import six.moves.urllib as urllib
import tarfile
from PIL import Image
from matplotlib import pyplot as plt
from  object_detection.utils import label_map_util
from  object_detection.utils import visualization_utils as vis_util
import cv2


print (tf.__version__)
if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

C:\Users\rizky\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.5.1


C:\Users\rizky\Documents\dsw\models\research\object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\rizky\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\rizky\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\rizky\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\rizky\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\rizky\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start

--------------------------------------------------------------------
## Tahap 2 : Download Model 

Periksa model di dalam direktori yang kita tuju. Jika belum ada, maka model akan di download di.[detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) 


In [2]:
 '''Download model'''
        
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
NUM_CLASSES = 90

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'


if not os.path.isfile(os.path.join(os.getcwd(),PATH_TO_CKPT)):   

    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        print (file)

        file_name = os.path.basename(file.name)
        print (file_name)
        if 'frozen_inference_graph.pb' in file_name:
          
            tar_file.extract(file, os.getcwd())
else:
    print ('Model sudah ada')
    
print ('model path :', os.path.join(os.getcwd(),PATH_TO_CKPT))

Model sudah ada
model path : C:\Users\rizky\Documents\dsw\dsw_testing\ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb


## Tahap 3 : Download Label

Jika kita sudah men-download repo object_detection tensorflow, kita sudah diberikan file label secara default untuk general model seperti data coco, kitti, pascal. Namun jika kita ingin kita ingin men-downloadnya langsung di repo tensorflow, bisa akses dengan cara sebagai berikut.

In [3]:
'''Download label '''

# LABEL URL
__LABEL_DOWNLOAD_URL = (
        'https://raw.githubusercontent.com/tensorflow/models/master/'
        'research/object_detection/data/mscoco_label_map.pbtxt')
# List of the strings that is used to add correct label for each box.
__LABEL_PATH = 'mscoco_label_map.pbtxt'

label_path = os.path.join(os.getcwd(),__LABEL_PATH)

if not os.path.exists(label_path):   
    opener = urllib.request.URLopener()
    opener.retrieve(__LABEL_DOWNLOAD_URL, label_path)

print ('Label Path : ',label_path)
print ('download selesai')

Label Path :  C:\Users\rizky\Documents\dsw\dsw_testing\mscoco_label_map.pbtxt
download selesai


In [4]:
'''
Bagian ini untuk mendeteksi muka, setelah melakukan pelatihan.
------------------------------------------------------------
Note : Non-aktifkan jika belum/tidak digunakan
'''
###  path model
# PATH_TO_CKPT='/home/nodeflux/workshop/dsw/dsw_training/training/frozen_inference_graph.pb'

###  path label
# __LABEL_PATH='/home/nodeflux/workshop/dsw/dsw_training/wider_label_map.pbtxt'

### jumlah class yang dideteksi
# NUM_CLASSES = 1

'\nBagian ini untuk mendeteksi muka, setelah melakukan pelatihan.\n------------------------------------------------------------\nNote : Non-aktifkan jika belum/tidak digunakan\n'

## Tahap 4 : Inisialisasi memory dan model tensorflow (frozen grap)

In [5]:
'''Load Model into memory'''

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

# Definite input and output Tensors for detection_graph
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


----------------------------------------
## Tahap 5 : Inisialisasi Label

In [6]:
''' Load Label into memory'''

label_map = label_map_util.load_labelmap(__LABEL_PATH)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

print (category_index)
print ('--done--')

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name'

## Tahap 6 : Objek Deteksi dengan Gambar 

In [1]:
'''
Bagian ini untuk mendeteksi satu gambar
------------------------------------------------------------
Note : Non-aktifkan jika belum/tidak digunakan
'''
# def image_pil_to_np(image):
#   (im_width, im_height) = image.size
#   return np.array(image.getdata()).reshape(
#       (im_height, im_width, 3)).astype(np.uint8)

# # Load an image (path): 
# image_path='1.jpg'
# img_input=cv2.imread(image_path) #open cv

# with detection_graph.as_default():
#     with tf.Session(graph=detection_graph) as sess:
#         # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
#         image_np_expanded = np.expand_dims(img_input, axis=0)       
        
#         # Actual detection.
#         (boxes, scores, classes, num) = sess.run(
#               [detection_boxes, detection_scores, detection_classes, num_detections],
#               feed_dict={image_tensor: image_np_expanded})
#         # Visualization of the results of a detection.
#         vis_util.visualize_boxes_and_labels_on_image_array(
#                                                           img_input,
#                                                           np.squeeze(boxes),
#                                                           np.squeeze(classes).astype(np.int32),
#                                                           np.squeeze(scores),
#                                                           category_index,
#                                                           min_score_thresh=.35,
#                                                           use_normalized_coordinates=True,
#                                                           line_thickness=1)
        
#         sess.close()
#         #### jika munggunakan open cv##############
#         cv2.imshow('preview',img_input)
#         cv2.waitKey(0)        

#         print ('The result of detection object')
#         print ('----boxes-----')
#         print (boxes)
#         print ('----scores-----')
#         print (np.squeeze(scores))
#         print ('----classes-----')
#         print (classes)
#         print ('----num-----')
#         print (num)       


# cv2.destroyAllWindows()
# print ('done')


'\nBagian ini untuk mendeteksi satu gambar\n------------------------------------------------------------\nNote : Non-aktifkan jika belum/tidak digunakan\n'

# Tahap 7 : Objek Deteksi Menggunakan Video

In [3]:
'''
Bagian ini untuk mendeteksi dari file stream video/ camera
------------------------------------------------------------
Note : Non-aktifkan jika belum/tidak digunakan
'''

cap =cv2.VideoCapture(0)

while (True):
    ret,frame= cap.read()
    img_input=frame
    
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(img_input, axis=0)       

            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                                                              img_input,
                                                              np.squeeze(boxes),
                                                              np.squeeze(classes).astype(np.int32),
                                                              np.squeeze(scores),
                                                              category_index,
                                                              min_score_thresh=.35,
                                                              use_normalized_coordinates=True,
                                                              line_thickness=1)
    
    
        cv2.imshow('preview',img_input)
        cv2.waitKey(1)
cv2.destroyAllWindows()
    

NameError: name 'cv2' is not defined